In [1]:
import pandas as pd
import numpy as np

from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../../data/tlp/synthetic-tvaes.csv')

In [3]:
drop_columns = [
    'ISC.resultadosCovid.numDosis',
    'ISC.resultadosCovid.numVecesCovid',
    'ISC.resultadosCovid.numVecesCovid_mode',
    'ISC.resultadosCovid.pasadoCovid',
    'ISC.resultadosCovid.sentimientoAislado',
    'ISC.resultadosCovid.sentimientoAislado_mediaFiltro',
    'ISC.resultadosCovid.sentimientoAislado_mode',
    'ISC.resultadosCovid.sintomaCansancio',
    'ISC.resultadosCovid.sintomaDiarrea',
    'ISC.resultadosCovid.sintomaDifRespirar',
    'ISC.resultadosCovid.sintomaDolCabeza',
    'ISC.resultadosCovid.sintomaDolGarganta',
    'ISC.resultadosCovid.sintomaDolMuscular',
    'ISC.resultadosCovid.sintomaEscalofrios',
    'ISC.resultadosCovid.sintomaFiebre',
    'ISC.resultadosCovid.sintomaOtro',
    'ISC.resultadosCovid.sintomaOtroString',
    'ISC.resultadosCovid.sintomaPerGusto',
    'ISC.resultadosCovid.sintomaPerOlfato',
    'ISC.resultadosCovid.vacunado',
    'ISC.resultadosObservacion.añosSinFumar',
    'ISC.resultadosObservacion.añosSinFumar_Rangos',
    'ISC.resultadosObservacion.consumoAlcohol_UBEsemanal',
    'ISC.resultadosObservacion.consumoAlcohol_UBEsemanal_knn',
    'ISC.resultadosObservacion.consumoAlcohol_UBEsemanal_mode&rangoEdad',
    'ISC.resultadosObservacion.enfermedadesAltTiroides',
    'ISC.resultadosObservacion.enfermedadesAntecedentesCardiacos',
    'ISC.resultadosObservacion.enfermedadesCardiopatiaIsquemica',
    'ISC.resultadosObservacion.enfermedadesHepatica',
    'ISC.resultadosObservacion.enfermedadesNinguna',
    'ISC.resultadosObservacion.enfermedadesOtros',
    'ISC.resultadosObservacion.enfermedadesRenal',
    'ISC.resultadosObservacion.entorno',
    'ISC.resultadosObservacion.familiaresAlzheimer',
    'ISC.resultadosObservacion.familiaresOtraDemencia',
    'ISC.resultadosObservacion.fumador',
    'ISC.resultadosObservacion.numCigarros',
    'ISC.resultadosObservacion.numCigarrosDia_knn',
    'ISC.resultadosObservacion.numCigarrosDia_mode&rangoEdad',
    'ISC.resultadosObservacion.tratCronico',
    'ISC.resultadosObservacion.tratPsicologic',
    'ISC.resultadosObservacion.tratPsiquiatrico',
    'ISC.resultadosOtros.frecOlvidos',
    'ISC.resultadosOtros.frecUsoEmail',
    'ISC.resultadosOtros.frecUsoMovil',
    'ISC.resultadosOtros.frecUsoOrdenador',
    'ISC.resultadosOtros.frecUsoRSociales',
    'ISC.resultadosOtros.frecUsoTele',
    'ISC.resultadosOtros.nivelActFisica',
    'ISC.resultadosOtros.nivelSociabilidad',
    'ISC.resultadosReservaCognitiva.actividadLectora',
    'ISC.resultadosReservaCognitiva.cursos',
    'ISC.resultadosReservaCognitiva.escolaridadPadres',
    'ISC.resultadosReservaCognitiva.formacionMusical',
    'ISC.resultadosReservaCognitiva.juegos'
]

In [4]:
pt_columns = [column for column in df.columns if 'PT' in column ]
pz_columns = [column for column in df.columns if 'PZ' in column ]
pdc_columns = [column for column in df.columns if 'PDC' in column]
drop_columns.extend(pt_columns)
drop_columns.extend(pz_columns)
drop_columns.extend(pdc_columns)

In [5]:
df.drop(columns=drop_columns, inplace=True)
df.replace([np.inf, -np.inf], 0, inplace=True)
df.head()

,ED_2Clases,ISC.FPacientes.genero,ISC.FPacientes.Edad,ISC.resultadosCribaje.Total.PD,ISC.resultadosObservacion.alcohol,ISC.resultadosObservacion.ansiedad,ISC.resultadosObservacion.depresion,ISC.resultadosObservacion.enfermedadesDiabetes,ISC.resultadosObservacion.enfermedadesHipercolesterolemia,ISC.resultadosObservacion.enfermedadesHipertension,...,TLP.Tavec.23_FP.PD,TLP.Tavec.24_Discriminabilidad.PD,TLP.Tavec.25_Sesgo.PD,TLP.Tavec.26_RI-B_frente_RI-A1.PD,TLP.Tavec.27_RL-CP_frente_RI-A5.PD,TLP.Tavec.28_RCl-Cp_frente_RCl-LP.PD,TLP.Tavec.29_RL-LP_frente_RL-CP.PD,TLP.Tavec.30_RCl-LP_frente_RL-LP.PD,TLP.Tavec.31_Recon-Ac_frente_RL-LP.PD,TLP.Tavec.32_Recon-Ac_frente_RCl-LP.PD
0,D,2,82,16,0,0,1,1,1,0,...,4,84.090909,0.142857,-20.000000,0.000000,33.333333,37.500000,-33.333333,-38.461538,-7.692308
1,D,2,75,16,0,1,1,0,0,0,...,3,79.545455,-0.333333,-71.428571,12.500000,0.000000,0.000000,0.000000,-20.000000,-20.000000
2,H,2,72,14,0,0,0,0,0,0,...,1,88.636364,-0.600000,-14.285714,75.000000,25.000000,-20.000000,0.000000,-16.666667,-16.666667
3,D,2,60,11,0,0,0,0,0,0,...,4,90.909091,0.600000,60.000000,16.666667,0.000000,50.000000,-20.000000,-50.000000,-37.500000
4,D,2,64,14,0,0,0,0,0,0,...,0,86.363636,-0.714286,0.000000,66.666667,0.000000,-33.333333,0.000000,-10.000000,-10.000000


In [6]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.shape, df_test.shape

((376, 68), (94, 68))

In [7]:
predictor= TabularPredictor(
    label='ED_2Clases',
    problem_type='binary',
    eval_metric='roc_auc',
    sample_weight='balance_weight',
    path='AutogluonModels/synthetic/tvaes/v1'
    )

In [8]:
predictor.fit(
    train_data = df_train,
    presets = ['high_quality'],
    time_limit = 300,
    auto_stack = True,
    excluded_model_types=['KNN','RF','XT', 'LR'],
    verbosity = 2
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       6.57 GB / 15.94 GB (41.2%)
Disk Space Avail:   84.52 GB / 446.36 GB (18.9%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and 

(_ray_fit pid=8372) [1000]	valid_set's binary_logloss: 0.111847


(_dystack pid=15912) 	0.9431	 = Validation score   (roc_auc)
(_dystack pid=15912) 	1.4s	 = Training   runtime
(_dystack pid=15912) 	0.02s	 = Validation runtime
(_dystack pid=15912) Fitting model: LightGBM_BAG_L1 ... Training model for up to 38.06s of the 60.73s of remaining time.
(_dystack pid=15912) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.16%)
(_dystack pid=15912) 	0.9279	 = Validation score   (roc_auc)
(_dystack pid=15912) 	0.92s	 = Training   runtime
(_dystack pid=15912) 	0.01s	 = Validation runtime
(_dystack pid=15912) Fitting model: CatBoost_BAG_L1 ... Training model for up to 33.75s of the 56.42s of remaining time.
(_dystack pid=15912) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.31%)
(_dystack pid=15912) 	0.9369	 = Validation score   (roc_auc)
(_dystack pid=15912) 	18.84s	 = Training   runtime
(_dystack pid=159

In [9]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                               model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0                WeightedEnsemble_L3   0.956738     roc_auc       0.346753  36.544295                0.001000           0.039837            3      False         24
1                WeightedEnsemble_L2   0.956003     roc_auc       0.176505  14.386214                0.000000           0.041019            2      False         15
2                    CatBoost_BAG_L2   0.954418     roc_auc       0.252413  28.102160                0.026563           7.534491            2      False         18
3               CatBoost_r177_BAG_L2   0.952013     roc_auc       0.273000  27.769133                0.047150           7.201464            2      False         22
4                     XGBoost_BAG_L2   0.949580     roc_auc       0.255641  22.582685                0.029791         

{'model_types': {'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'CatBoost_r9_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r96_BAG_L1': 'StackerEnsembleModel_LGB',
  'XGBoost_r33_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'CatBoost_r137_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetFastAI_r102_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'LightGBMXT_BAG_L2': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L2': 'StackerEnsembleModel_LGB',
  'CatBoost_BAG_L2': 'S

In [10]:
auxiliary_metrics = ['balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'roc_auc', 'average_precision', 'precision', 'recall', 'log_loss', 'pac_score']

In [11]:
predictor.evaluate(
    df_test,
    silent = False,
    auxiliary_metrics = auxiliary_metrics
    )

Evaluation: roc_auc on test data: 0.9651268115942029
Evaluations on test data:
{
    "roc_auc": 0.9651268115942029,
    "accuracy": 0.925531914893617,
    "balanced_accuracy": 0.9257246376811594,
    "mcc": 0.8512565307587486,
    "f1": 0.9263157894736842,
    "precision": 0.9361702127659575,
    "recall": 0.9166666666666666
}


{'roc_auc': 0.9651268115942029,
 'accuracy': 0.925531914893617,
 'balanced_accuracy': 0.9257246376811594,
 'mcc': 0.8512565307587486,
 'f1': 0.9263157894736842,
 'precision': 0.9361702127659575,
 'recall': 0.9166666666666666}

In [12]:
predictor.leaderboard(
    df_train,
    extra_metrics = auxiliary_metrics,
    silent=True
)

,model,score_test,balanced_accuracy,f1,f1_macro,f1_micro,roc_auc,average_precision,precision,recall,...,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_r177_BAG_L1_FULL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.004622,NaN,0.349450,0.004622,NaN,0.349450,1,True,31
1,CatBoost_BAG_L1_FULL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.014718,NaN,0.827810,0.014718,NaN,0.827810,1,True,27
2,CatBoost_r9_BAG_L1_FULL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.015844,NaN,2.008223,0.015844,NaN,2.008223,1,True,34
3,CatBoost_r137_BAG_L1_FULL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.016663,NaN,0.401208,0.016663,NaN,0.401208,1,True,37
4,LightGBM_BAG_L1_FULL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.016704,NaN,0.104872,0.016704,NaN,0.104872,1,True,26
5,XGBoost_r33_BAG_L1_FULL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.016714,NaN,0.380683,0.016714,NaN,0.380683,1,True,36
6,LightGBM_r96_BAG_L1_FULL,1.000000,0.997354,0.997333,0.997340,0.997340,1.000000,1.000000,0.994681,1.000000,...,roc_auc,0.022092,NaN,0.303133,0.022092,NaN,0.303133,1,True,35
7,LightGBMXT_BAG_L1_FULL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.022655,NaN,0.364641,0.022655,NaN,0.364641,1,True,25
8,LightGBMLarge_BAG_L1_FULL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.024616,NaN,0.201422,0.024616,NaN,0.201422,1,True,30
9,XGBoost_BAG_L1_FULL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,roc_auc,0.027438,NaN,0.174384,0.027438,NaN,0.174384,1,True,29


In [13]:
predictor.feature_importance(data=df_train)

Computing feature importance via permutation shuffling for 67 features using 376 rows with 5 shuffle sets...
	35.75s	= Expected runtime (7.15s per shuffle set)
	8.15s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
TLP.Stroop.color.PD,0.014096,0.002160,0.000064,5,0.018544,0.009648
TLP.CubCorsi.Directos.PD,0.000124,0.000055,0.003623,5,0.000238,0.000011
ISC.FPacientes.Edad,0.000051,0.000024,0.004290,5,0.000100,0.000002
ISC.FPacientes.genero,0.000000,0.000000,0.500000,5,0.000000,0.000000
TLP.Tavec.09_RL-LP.PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
...,...,...,...,...,...,...
TLP.MMSE.Memoria.PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
TLP.MMSE.Orientacion.PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
TLP.MMSE.Total.PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
TLP.Stroop.interferencia.PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
